# 1. ollama exaone 설치

In [ ]:
!pip install colab-xterm

!pip install langchain langgraph langchain-community chromadb sqlite-utils
curl -fsSL https://ollama.com/install.sh | sh

올라마 서버 실행

In [ ]:
ollama serve & ollama pull exaone3.5:2.4b

올라마 종료

In [ ]:
!ps aux | grep ollama
# ollama   1360934  0.4  0.6 11169072 209564 ?     Ssl  11:33   0:57 /usr/local/bin/ollama serve

# !sudo systemctl stop ollama

# Mysql 접속

라이브러리 설치

In [ ]:
!pip install mysql-connector-python

Mysql 사용자 정보 입력

In [ ]:
import mysql.connector

# MySQL 접속 정보 설정
# !!! 이 부분을 실제 사용자 정보로 변경해야 합니다. !!!
DB_CONFIG = {
    "host": "localhost",   # MySQL 서버 주소 (로컬인 경우 'localhost')
    "user": "admin", # MySQL 사용자 이름
    "password": "1qazZAQ!", # MySQL 비밀번호
    "database": "final" # 사용할 데이터베이스 이름 (미리 생성되어 있어야 함)
}

Mysql 접속 확인

In [ ]:
def check_mysql_connection():
    """MySQL 데이터베이스 연결을 시도하고 결과를 출력하는 함수"""
    print(f"[{DB_CONFIG['user']}@{DB_CONFIG['host']}] 로 MySQL 연결을 시도합니다...")
    
    try:
        # DB 연결 시도
        conn = mysql.connector.connect(
            host=DB_CONFIG["host"],
            user=DB_CONFIG["user"],
            password=DB_CONFIG["password"],
            # database=DB_CONFIG["database"] # 특정 DB 없이 서버 접속만 확인할 경우 주석 처리
        )
        
        # 연결 성공
        print("\n🎉 **MySQL 접속 성공!**")
        print(f"서버 버전: {conn.get_server_info()}")
        
        # 연결 종료
        conn.close()

    except mysql.connector.Error as err:
        # 연결 실패 시 오류 처리
        print("\n❌ **MySQL 접속 실패!**")
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("오류: 사용자 이름 또는 비밀번호가 잘못되었습니다. (Access Denied)")
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print(f"오류: 데이터베이스 '{DB_CONFIG['database']}'가 존재하지 않습니다.")
        else:
            print(f"알 수 없는 오류 발생: {err}")
    
# 함수 실행
check_mysql_connection()

패키지 설치

In [ ]:
!pip install pymysql langchain-community langchain-text-splitters langchain-core

# 벡터스토어 만들기

In [ ]:
import pymysql
from langchain_core.documents import Document

# 변경: langchain.text_splitter → langchain_text_splitters
from langchain_text_splitters import CharacterTextSplitter  # 이 부분이 핵심 수정!

from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

# MySQL 접속 정보
DB_CONFIG = {
    'host': 'localhost',
    'user': 'admin',
    'password': '1qazZAQ!',
    'db': 'final',
    'charset': 'utf8mb4'
}

def build_rag_chroma():
    # MySQL의 documents 테이블에서 데이터를 로드하여 Chroma 벡터스토어를 생성하고 저장합니다.
    conn = None
    documents = []

    try:
        # MySQL 연결
        conn = pymysql.connect(**DB_CONFIG)
        print("✅ MySQL 연결 성공")

        with conn.cursor() as cursor:
            # title + summary + id 조회로 변경
            cursor.execute("SELECT id, title, summary FROM documents")
            rows = cursor.fetchall()

            if not rows:
                print("⚠️ 로드된 데이터가 없습니다. 'documents' 테이블을 확인해주세요.")
                return

            # Document 객체로 변환 (page_content = title + summary)
            for row in rows:
                doc_id = row[0]
                title_text = (row[1] or "").strip()
                summary_text = (row[2] or "").strip()

                # title + summary 결합 (둘 중 하나만 있어도 안전하게 처리)
                if title_text and summary_text:
                    combined_text = f"{title_text}. {summary_text}"
                elif title_text:
                    combined_text = title_text
                else:
                    combined_text = summary_text  # summary만 있어도 진행

                if not combined_text:
                    # 둘 다 비어있으면 스킵
                    continue

                doc = Document(
                    page_content=combined_text,
                    metadata={
                        "source": "mysql",
                        "table": "documents",
                        "id": doc_id,
                        "title": title_text
                    }
                )
                documents.append(doc)

            print(f"✅ MySQL에서 {len(documents)}개 문서 로드 완료")

            # 로드된 문서 미리보기 출력
            for i, doc in enumerate(documents[:5]):  # 너무 많을 수 있으니 상위 5개만
                print(f"\n--- 문서 #{i + 1} (ID: {doc.metadata.get('id', 'N/A')}) ---")
                print(f"  Title: {doc.metadata.get('title', '')}")
                print(f"  Metadata: {doc.metadata}")
                print(f"  Content (일부): {doc.page_content[:200]}...")

    except pymysql.Error as err:
        print(f"❌ MySQL 오류: {err}")
        return
    finally:
        if conn:
            conn.close()
            print("🔒 MySQL 연결 해제")

    if not documents:
        print("⚠️ 유효한 문서가 없어 벡터스토어를 생성하지 않습니다.")
        return

    # 텍스트 분할 (청킹)
    splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
    split_docs = splitter.split_documents(documents)
    print(f"\n✅ 청킹 완료. 총 {len(split_docs)}개 청크 생성")

    # 임베딩 모델 설정
    try:
        embeddings = OllamaEmbeddings(model="exaone3.5:2.4b")  # 임베딩 전용 모델 권장: nomic-embed-text 등
        print("✅ 임베딩 모델 설정 완료")
    except Exception as e:
        print(f"❌ 임베딩 모델 설정 오류: {e}")
        print("   Ollama 서버가 실행 중인지, 임베딩 가능한 모델인지 확인하세요.")
        return

    # Chroma 벡터스토어 생성 및 저장
    print("\n⏳ 벡터스토어 생성 및 임베딩 중...")
    rag_path = "./rag_chroma/documents/title_summary/"
    
    db = Chroma.from_documents(
        documents=split_docs,
        embedding=embeddings,
        persist_directory=rag_path
    )
    db.persist()
    
    print(f"🎉 RAG Chroma 벡터스토어 구축 완료!")
    print(f"   저장 경로: {rag_path}")

# 실행
build_rag_chroma()


# DB 리스트 불러오기

In [48]:
import pymysql # MySQL 접속 라이브러리 임포트

DB_CONFIG = {
    'host': 'localhost', 
    'user': 'admin', 
    'password': '1qazZAQ!', 
    'db': 'final',
    'charset': 'utf8mb4'
}

def query_documents_db_mysql():
    conn = None # 연결 객체 초기화
    
    try:
        # 1. MySQL 서버에 연결 (DB_CONFIG 사용)
        conn = pymysql.connect(**DB_CONFIG)
        cursor = conn.cursor() # 커서 생성

        # 2. 데이터 조회
        cursor.execute("SELECT * FROM documents")
        documents = cursor.fetchall() # 모든 결과 가져오기

        # 3. 데이터 출력
        print("\n---documents Table Data (MySQL)---")
        for row in documents:
            print(f"id: {row[0]}, title: {row[1]}, file_location: {row[4]}...")

    except pymysql.Error as err:
        print(f"❌ MySQL 작업 중 오류 발생: {err}")
        
    finally:
        # 4. 연결 해제
        if conn:
            conn.close()

# 함수 실행
query_documents_db_mysql()


---documents Table Data (MySQL)---
id: 168, title: 이재명 대통령, 국군의 날 기념행사 주재 및 국민군대 개념 강조, file_location: fileList/국민의군대.docx...
id: 169, title: **기초생활수급자 등 취약계층 대상 무료 신문 구독 지원 사업 안내**, file_location: fileList/2025년 신문 구독 지원 신청 전 필독사항.pdf...
id: 170, title: 국가정보자원관리원 화재 원인 조사 중 4명 입건, file_location: fileList/사회기사정보.docx...
id: 171, title: 주차 갈등으로 특수폭행 고소당한 사례, file_location: fileList/파렴치한.docx...
id: 172, title: 이재명 대통령, 국민참여 국군의 날 행사 주재, file_location: fileList/국민의군대.docx...
id: 173, title: 이재명 대통령의 청년주권시대 청년정책 추진, file_location: fileList/소름돋는정책원해.docx...
id: 174, title: 전유성, 코미디계 '전설'로 떠난 76세 노인, file_location: fileList/전유성별세.docx...
id: 175, title: 아틀레티코 마드리드의 메이슨 그린우드 영입 추진, file_location: fileList/해외축구_기사.docx...
id: 176, title: 한국 9월 수출 역대 최대치 경신: 반도체와 자동차 주도 성장, file_location: fileList/경제기사.docx...
id: 177, title: 윤석열 전 대통령 교정수발 논란 폭로, 감찰 착수, file_location: fileList/랭킹뉴스.docx...
id: 178, title: 엘리트 청년의 노숙자 생활 선언: 학문적 성취 뒤에 숨겨진 고독과 선택, file_location: fileList/세계기사.docx...
id: 179, 

추가 패키지 설치

In [50]:
!pip install grandalf

  Using cached grandalf-0.8-py3-none-any.whl.metadata (1.7 kB)
  Using cached pyparsing-3.2.5-py3-none-any.whl.metadata (5.0 kB)
Using cached grandalf-0.8-py3-none-any.whl (41 kB)
Using cached pyparsing-3.2.5-py3-none-any.whl (113 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [grandalf]


In [7]:
import pymysql
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from typing import List, Dict, Any

# MySQL 접속 정보
DB_CONFIG = {
    'host': 'localhost',
    'user': 'admin',
    'password': '1qazZAQ!',
    'db': 'final',
    'charset': 'utf8mb4'
}

# Chroma 설정
CHROMA_PATH = "./rag_chroma/documents/title_summary/"
EMBEDDINGS = OllamaEmbeddings(model="exaone3.5:2.4b")

def search_rag(query: str, top_k: int = 5) -> List[Dict[str, Any]]:
    # 1. Chroma 벡터스토어 로드
    try:
        vectorstore = Chroma(persist_directory=CHROMA_PATH, embedding_function=EMBEDDINGS)
        print("✅ Chroma 벡터스토어 로드 완료")
    except Exception as e:
        print(f"❌ Chroma 로드 실패: {e}")
        return []

    # 2. 벡터스토어 검색 (유사도 포함)
    results = vectorstore.similarity_search_with_score(query, k=top_k * 2)  # 중복 제거를 위해 k 확대
    
    if not results:
        print("⚠️ 검색 결과 없음")
        return []

    # 3. 문서 ID로 그룹화 (최고 유사도 유지)
    doc_scores = {}
    for doc, score in results:
        doc_id = doc.metadata.get("id")
        if doc_id:
            if doc_id not in doc_scores or score < doc_scores[doc_id]:
                doc_scores[doc_id] = score  # 낮은 score = 더 높은 유사도
    
    if not doc_scores:
        return []

    # 4. 유사도 정규화 (0-100%로 변환)
    scores = list(doc_scores.values())
    min_score = min(scores)
    max_score = max(scores)
    
    # 5. MySQL에서 파일 메타데이터 조회
    conn = None
    final_results = []
    try:
        conn = pymysql.connect(**DB_CONFIG)
        cursor = conn.cursor(pymysql.cursors.DictCursor)
        
        # 문서 ID별로 MySQL 데이터 조회
        for doc_id, score in doc_scores.items():
            cursor.execute(
                "SELECT * FROM documents WHERE id = %s", 
                (doc_id,)
            )
            row = cursor.fetchone()
            if not row:
                continue
                
            # 유사도 % 계산
            if min_score == max_score:
                relevance = 100.0
            else:
                # 낮은 score = 높은 유사도 (0~100% 변환)
                relevance = round((1 - (score - min_score) / (max_score - min_score)) * 100, 1)
            
            # 결과 포맷
            final_results.append({
                "relevance": relevance,
                "file_name": row["file_name"],
                "file_location": row["file_location"],
                "summary": row["summary"][:200] + "..." if row["summary"] else "요약 없음",
                "title": row["title"],
                "created_at": row["created_at"].strftime("%Y-%m-%d") if row["created_at"] else "N/A",
                "doc_type": row["doc_type"]
            })
            
        # 결과 내림차순 정렬 (관련성 순)
        final_results.sort(key=lambda x: x["relevance"], reverse=True)
        final_results = final_results[:top_k]  # 상위 top_k개
        
    except Exception as e:
        print(f"❌ MySQL 오류: {e}")
    finally:
        if conn:
            conn.close()
    
    return final_results

# 실행 예시
if __name__ == "__main__":
    query = input("검색어를 입력하세요: ")
    results = search_rag(query, top_k=10)
    
    if not results:
        print("\n🔍 관련된 파일을 찾을 수 없습니다.")
    else:
        print(f"\n🔍 '{query}'에 대한 검색 결과 ({len(results)}건):")
        for i, res in enumerate(results, 1):
            print(f"\n--- {i}순위 ({res['relevance']}%) ---")
            print(f"📄 파일명: {res['file_name']}")
            print(f"📁 위치: {res['file_location']}")
            print(f"📝 요약: {res['summary']}")
            print(f"🏷️ 유형: {res['doc_type']}")
            print(f"🕒 등록일: {res['created_at']}")

✅ Chroma 벡터스토어 로드 완료

🔍 '마드리드'에 대한 검색 결과 (10건):

--- 1순위 (100.0%) ---
📄 파일명: 해외축구_기사.docx
📁 위치: fileList/해외축구_기사.docx
📝 요약: 아틀레티코 마드리드는 잉글랜드 출신 공격수 메이슨 그린우드를 영입하려는 의지를 보이며, 마르세유와는 이적료 약 1225억원으로 협상 중이다. 그린우드는 맨체스터 유나이티드 유스 시절부터 두각을 나타내다가 2021년 이후 법적 문제로 인해 위기를 겪었으나, 이후 마르세유로 이적해 리그앙 득점왕에 오르며 다시 주목받았다. 현재 그린우드의 기세는 이어지고 있어,...
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

--- 2순위 (82.8%) ---
📄 파일명: 세계기사.docx
📁 위치: fileList/세계기사.docx
📝 요약: 중국 상하이 출신의 금융학 석사 출신 청년 자오덴은 과도한 가족 압박과 외로움으로 인해 뉴질랜드 이주 후 다양한 도시에서 생활하며 학문적 성공을 이루었으나, 현재는 한 달에 약 2만원으로 생활하는 노숙자로 변모했다. 그는 깊이 느끼는 외로움과 부모와의 갈등 속에서 극단적인 저비용 생활을 선택해 자유를 추구하며, 의미 있는 활동으로 삶의 가치를 찾고 있다. ...
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

--- 3순위 (73.6%) ---
📄 파일명: 속도미달.docx
📁 위치: fileList/속도미달.docx
📝 요약: 서울시는 한강버스의 해상 시운전 결과를 알면서도 평균속력과 최대속력을 과장해 공개하며 시민들을 기만한 것으로 드러났다. 실제로 시운전에서 평균 최고속도는 15노트 미만으로, 계획된 17노트보다 낮게 나타났다. 서울시는 정식 운행 전 속도 조정에 대한 명확한 설명을 내놓지 않아 비판을 받았고, 이러한 문제점에도 불구하고 hurriedly 공식 발표를 진행하며...
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

--- 4순위 (64.3%) ---
📄 파일명: IT기사.

# 완전한 RAG

### 시스템 아키텍처
graph LR    
    A[사용자 질문] --> B(Retrieval)    
    B --> C[Chroma Vector DB]   
    C --> D[유사 문서 검색]    
    D --> E[MySQL 메타데이터]   
    E --> F[Augmentation]   
    F --> G[컨텍스트 결합]   
    G --> H[Generation]   
    H --> I[LLM 답변 생성]   
    I --> J[최종 출력]   

In [2]:
import pymysql
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from typing import List, Dict, Any, Optional

# MySQL 접속 정보
DB_CONFIG = {
    'host': 'localhost',
    'user': 'admin',
    'password': '1qazZAQ!',
    'db': 'final',
    'charset': 'utf8mb4'
}

# Chroma 설정
CHROMA_PATH = "./rag_chroma/documents/title_summary/"
EMBEDDINGS = OllamaEmbeddings(model="exaone3.5:2.4b")
LLM = ChatOllama(model="exaone3.5:2.4b", temperature=0.1)  # 생성 단계 LLM

def search_rag(query: str, top_k: int = 5) -> List[Dict[str, Any]]:
    # 1. Chroma 벡터스토어 로드
    try:
        vectorstore = Chroma(persist_directory=CHROMA_PATH, embedding_function=EMBEDDINGS)
        print("✅ Chroma 벡터스토어 로드 완료")
    except Exception as e:
        print(f"❌ Chroma 로드 실패: {e}")
        return []

    # 2. 벡터스토어 검색 (유사도 포함)
    results = vectorstore.similarity_search_with_score(query, k=top_k * 2)  # 중복 제거를 위해 k 확대
    
    if not results:
        print("⚠️ 검색 결과 없음")
        return []

    # 3. 문서 ID로 그룹화 (최고 유사도 유지)
    best_doc_info = {}
    for doc, score in results:
        doc_id = doc.metadata.get("id")
        if doc_id:
            content = doc.page_content.strip()  # 실제 내용
            if not content:  # 내용이 빈 경우 스킵
                continue
            if doc_id not in best_doc_info or score < best_doc_info[doc_id][0]:
                best_doc_info[doc_id] = (score, content)

    if not best_doc_info:
        return []

    # 4. 유사도 정규화 (0-100%로 변환)
    scores = [score for score, _ in best_doc_info.values()]
    min_score = min(scores)
    max_score = max(scores)
    
    # 5. MySQL에서 파일 메타데이터 조회
    conn = None
    final_results = []
    try:
        conn = pymysql.connect(**DB_CONFIG)
        cursor = conn.cursor(pymysql.cursors.DictCursor)
        
        # 문서 ID별로 MySQL 데이터 조회
        for doc_id, (score, content) in best_doc_info.items():
            cursor.execute(
                "SELECT * FROM documents WHERE id = %s", 
                (doc_id,)
            )
            row = cursor.fetchone()
            if not row:
                continue
                
            # 유사도 % 계산
            if min_score == max_score:
                relevance = 100.0
            else:
                # 낮은 score = 높은 유사도 (0~100% 변환)
                relevance = round((1 - (score - min_score) / (max_score - min_score)) * 100, 1)
            
            # 결과 포맷 (CONTENT 추가)
            final_results.append({
                "relevance": relevance,
                "file_name": row["file_name"],
                "file_location": row["file_location"],
                "summary": row["summary"][:200] + "..." if row["summary"] else "요약 없음",
                "title": row["title"],
                "created_at": row["created_at"].strftime("%Y-%m-%d") if row["created_at"] else "N/A",
                "doc_type": row["doc_type"],
                "content": content  # Chroma에서 가져온 실제 내용
            })
            
        # 결과 내림차순 정렬 (관련성 순)
        final_results.sort(key=lambda x: x["relevance"], reverse=True)
        final_results = final_results[:top_k]  # 상위 top_k개
        
    except Exception as e:
        print(f"❌ MySQL 오류: {e}")
    finally:
        if conn:
            conn.close()
    
    return final_results

def generate_answer(query: str, context: str) -> str:
    """LLM을 사용해 쿼리에 대한 답변 생성"""
    try:
        # 1. 프롬프트 템플릿 정의
        prompt = ChatPromptTemplate.from_template(
            """다음 문서 내용들을 바탕으로 사용자 질문에 답변해 주세요.
            - 문서에 직접 언급된 내용만을 바탕으로 답변해야 합니다.
            - 정보가 없는 경우 "정보가 없습니다"라고 명확히 답변해 주세요.
            - 항상 공손하고 전문적인 어조를 유지해 주세요.

            ### 문서 내용:
            {context}

            ### 사용자 질문:
            {query}

            ### 답변: 찾은 문서중에 몇번째 정보가 가장 관련이 높은지를 상세히 답변해 주세요. (몇번째, 무슨파일인지, 어디에 있는지, 요약은 무엇인지 등)
            """
            
            
        )
        
        # 2. LLM 호출
        chain = prompt | LLM | StrOutputParser()
        result = chain.invoke({"context": context, "query": query})
        return result.strip()
    except Exception as e:
        print(f"❌ 생성 오류: {e}")
        return "답변을 생성하는 중에 오류가 발생했습니다."

# 실행 예시
if __name__ == "__main__":
    query = input("검색어를 입력하세요: ")
    results = search_rag(query, top_k=3)  # 3개의 문서 사용
    
    if not results:
        print("\n🔍 관련된 파일을 찾을 수 없습니다.")
    else:
        # 1. 검색 결과 표시
        print(f"\n🔍 '{query}'에 대한 검색 결과 ({len(results)}건):")
        for i, res in enumerate(results, 1):
            print(f"\n--- {i}순위 ({res['relevance']}%) ---")
            print(f"📄 파일명: {res['file_name']}")
            print(f"📁 위치: {res['file_location']}")
            print(f"📝 요약: {res['summary']}")
            print(f"🏷️ 유형: {res['doc_type']}")
            print(f"🕒 등록일: {res['created_at']}")
        
        # 2. Augmentation: 검색 결과를 하나의 컨텍스트로 결합
        context = "\n\n".join([res['content'] for res in results])
        print(f"\n🧠 AI가 분석할 문서 내용 ({len(context)}자):")
        print(f"['{'*'*50}']")  # 길이 확인용
        print(f"{context[:200]}{'...' if len(context)>200 else ''}")
        print(f"['{'*'*50}']")
        
        # 3. Generation: AI 답변 생성
        print("\n💬 AI가 생성한 답변:")
        answer = generate_answer(query, context)
        print(f"{answer}")

✅ Chroma 벡터스토어 로드 완료

🔍 '이재명'에 대한 검색 결과 (3건):

--- 1순위 (100.0%) ---
📄 파일명: 소름돋는정책원해.docx
📁 위치: fileList/소름돋는정책원해.docx
📝 요약: 이재명 정부는 청년주간을 선언하며 '국민주권정부 청년정책 추진방향'을 발표, 청년의 체감과 참여를 핵심으로 삼아 일자리 창출, 주거 지원 확대, 실질적 정책 참여 확대에 중점을 둔 139개 세부과제를 제시했다. 특히 청년 월세지원 사업의 확대와 대기업 신규 채용 확대를 통해 청년들의 일자리 문제를 해결하려는 노력을 보여주며, 이는 청년 세대의 가장 큰 어려...
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

--- 2순위 (50.5%) ---
📄 파일명: 국민의군대.docx
📁 위치: fileList/국민의군대.docx
📝 요약: 이재명 대통령이 건군 77주년 국군의 날 행사를 계룡대에서 주재하며, 국민과 함께하는 '국민의 군대' 정신을 재확인하고 첨단 군사 자산을 공개함으로써 대한민국 군대의 선진화와 위상 강화를 추구했다. 행사에서는 참전 유공자 훈장 수여와 함께 한국형 3축 체계와 K-방산 전력이 소개되어 군대의 현대화와 헌법 수호 의지를 분명히 나타냈다....
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

--- 3순위 (10.2%) ---
📄 파일명: 랭킹뉴스.docx
📁 위치: fileList/랭킹뉴스.docx
📝 요약: 윤석열 전 대통령이 특수공무집행방해 혐의로 수감 중일 때, 법무부 교정직원 7명이 그의 외부 활동을 돕는 등 부적절한 상황이 온라인 게시판을 통해 폭로되었다. 특히 미용사 초청, 무제한 변호사 접견 등이 문제로 제기되었으며, 이에 법무부는 감찰을 통해 책임 소재를 조사하고 있다. 사건은 교정 시스템의 공정성과 투명성에 대한 의문을 제기하며, 필요한 경우 예...
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

🧠 AI가 분석할 문서 내용 (808자):
['*****************

# RAG 구조로 변경

In [ ]:
pip install langchain langchain-community langchain-core langgraph pymysql chromadb ollama

In [2]:
from typing import TypedDict, List, Dict, Any
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langgraph.graph import StateGraph, END
import pymysql

# 상태 정의
class AgentState(TypedDict):
    query: str
    keywords: str
    search_results: List[Dict[str, Any]]
    context: str
    result: str

# DB 접속 정보
DB_CONFIG = {
    'host': 'localhost',
    'user': 'admin',
    'password': '1qazZAQ!',
    'db': 'final',
    'charset': 'utf8mb4'
}

# Chroma 설정
CHROMA_PATH = "./rag_chroma/documents/title_summary/"
EMBEDDINGS = OllamaEmbeddings(model="exaone3.5:2.4b")
LLM = Ollama(model="exaone3.5:2.4b")
CHAT_LLM = ChatOllama(model="exaone3.5:2.4b", temperature=0.1)

# 1. 키워드 추출 에이전트 (LLMChain 없이 직접 구현)
def extractor_agent(state: AgentState):
    keyword_prompt = PromptTemplate.from_template(
        """사용자의 질문에서 띄어쓰기 확인하고 찾고자 하는 키워드를 쉼표로 구분하여 출력하세요.
        벡터스토어 검색을 위한 최적의 키워드를 추출해주세요.
        \n질문: {query}"""
    )
    formatted_prompt = keyword_prompt.format(query=state["query"])
    keywords = LLM.invoke(formatted_prompt)
    return {**state, "keywords": keywords.strip()}

# 2. RAG 검색 에이전트
def rag_search_agent(state: AgentState):
    try:
        vectorstore = Chroma(persist_directory=CHROMA_PATH, embedding_function=EMBEDDINGS)
        results = vectorstore.similarity_search_with_score(state["keywords"], k=10)

        if not results:
            return {**state, "search_results": [], "context": ""}

        best_doc_info = {}
        for doc, score in results:
            doc_id = doc.metadata.get("id")
            if doc_id:
                content = doc.page_content.strip()
                if not content:
                    continue
                if doc_id not in best_doc_info or score < best_doc_info[doc_id][0]:
                    best_doc_info[doc_id] = (score, content)

        if not best_doc_info:
            return {**state, "search_results": [], "context": ""}

        scores = [score for score, _ in best_doc_info.values()]
        min_score = min(scores)
        max_score = max(scores)

        conn = None
        search_results = []
        try:
            conn = pymysql.connect(**DB_CONFIG)
            cursor = conn.cursor(pymysql.cursors.DictCursor)

            for doc_id, (score, content) in best_doc_info.items():
                cursor.execute("SELECT * FROM documents WHERE id = %s", (doc_id,))
                row = cursor.fetchone()
                if not row:
                    continue

                if min_score == max_score:
                    relevance = 100.0
                else:
                    relevance = round((1 - (score - min_score) / (max_score - min_score)) * 100, 1)

                search_results.append({
                    "relevance": relevance,
                    "file_name": row["file_name"],
                    "file_location": row["file_location"],
                    "summary": row["summary"][:200] + "..." if row["summary"] else "요약 없음",
                    "title": row["title"],
                    "created_at": row["created_at"].strftime("%Y-%m-%d") if row["created_at"] else "N/A",
                    "doc_type": row["doc_type"],
                    "content": content
                })

            search_results.sort(key=lambda x: x["relevance"], reverse=True)
            search_results = search_results[:5]
            context = "\n\n".join([res['content'] for res in search_results])

        except Exception as e:
            print(f"❌ MySQL 오류: {e}")
            return {**state, "search_results": [], "context": ""}
        finally:
            if conn:
                conn.close()

        return {**state, "search_results": search_results, "context": context}

    except Exception as e:
        print(f"❌ RAG 검색 오류: {e}")
        return {**state, "search_results": [], "context": ""}

# 3. 답변 생성 에이전트
def answer_generator_agent(state: AgentState):
    if not state["search_results"]:
        return {**state, "result": "관련 정보를 찾을 수 없습니다."}

    try:
        prompt = ChatPromptTemplate.from_template(
            """다음 문서 내용들을 바탕으로 사용자 질문에 답변해 주세요.
            - 문서에 직접 언급된 내용만을 바탕으로 답변해야 합니다.
            - 정보가 없는 경우 "정보가 없습니다"라고 명확히 답변해 주세요.
            - 항상 공손하고 전문적인 어조를 유지해 주세요.

            ### 검색 결과 요약:
            {search_summary}

            ### 문서 내용:
            {context}

            ### 사용자 질문:
            {query}

            ### 답변:
            찾은 문서중에 몇번째 정보가 가장 관련이 높은지를 상세히 답변해 주세요.
            (몇번째, 무슨파일인지, 어디에 있는지, 요약은 무엇인지 등)
            """
        )

        search_summary = "\n".join([
            f"{i+1}순위 ({res['relevance']}%): {res['file_name']} ({res['doc_type']}) - {res['summary']}"
            for i, res in enumerate(state["search_results"])
        ])

        chain = prompt | CHAT_LLM | StrOutputParser()
        result = chain.invoke({
            "search_summary": search_summary,
            "context": state["context"],
            "query": state["query"]
        })

        return {**state, "result": result.strip()}

    except Exception as e:
        print(f"❌ 답변 생성 오류: {e}")
        return {**state, "result": "답변을 생성하는 중에 오류가 발생했습니다."}

# 4. 결과 포맷팅 에이전트
def result_formatter_agent(state: AgentState):
    if not state["search_results"]:
        return state

    formatted_result = "\n🔍 검색 결과:\n"
    for i, res in enumerate(state["search_results"], 1):
        formatted_result += f"\n--- {i}순위 ({res['relevance']}%) ---\n"
        formatted_result += f"📄 파일명: {res['file_name']}\n"
        formatted_result += f"📁 위치: {res['file_location']}\n"
        formatted_result += f"📝 요약: {res['summary']}\n"
        formatted_result += f"🏷️ 유형: {res['doc_type']}\n"
        formatted_result += f"🕒 등록일: {res['created_at']}\n"

    formatted_result += f"\n💬 AI 답변:\n{state['result']}"
    return {**state, "result": formatted_result}

# LangGraph 연결
graph = StateGraph(AgentState)
graph.add_node("extractor", extractor_agent)
graph.add_node("rag_search", rag_search_agent)
graph.add_node("answer_generator", answer_generator_agent)
graph.add_node("result_formatter", result_formatter_agent)

graph.set_entry_point("extractor")
graph.add_edge("extractor", "rag_search")
graph.add_edge("rag_search", "answer_generator")
graph.add_edge("answer_generator", "result_formatter")
graph.add_edge("result_formatter", END)

app = graph.compile()

# 실행 예시
if __name__ == "__main__":
    query = input("검색어를 입력하세요: ")
    state = {
        "query": query,
        "keywords": "",
        "search_results": [],
        "context": "",
        "result": ""
    }
    result = app.invoke(state)
    print(result["result"])

/tmp/ipykernel_3453010/2139675930.py:48: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory=CHROMA_PATH, embedding_function=EMBEDDINGS)



🔍 검색 결과:

--- 1순위 (100.0%) ---
📄 파일명: IT기사.docx
📁 위치: fileList/IT기사.docx
📝 요약: 홍민택 카카오 최고제품책임자(CPO)가 사내 공지를 통해 카카오톡 개편으로 인한 사용자 불만에 사과하며, 기존 서비스 유지와 함께 친구탭 피드 노출 방식 변경 및 인스타그램식 콘텐츠 분리 배치 계획을 밝혔다. 트래픽 감소는 없다고 전망하면서도 사용자 불편 최소화에 초점을 맞추겠다고 강조했다. 개편 배경은 카카오의 성장 정체 상황에서의 혁신 필요성과 기존 메...
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

--- 2순위 (85.0%) ---
📄 파일명: 해외축구_기사.docx
📁 위치: fileList/해외축구_기사.docx
📝 요약: 아틀레티코 마드리드는 잉글랜드 출신 공격수 메이슨 그린우드를 영입하려는 의지를 보이며, 마르세유와는 이적료 약 1225억원으로 협상 중이다. 그린우드는 맨체스터 유나이티드 유스 시절부터 두각을 나타내다가 2021년 이후 법적 문제로 인해 위기를 겪었으나, 이후 마르세유로 이적해 리그앙 득점왕에 오르며 다시 주목받았다. 현재 그린우드의 기세는 이어지고 있어,...
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

--- 3순위 (63.1%) ---
📄 파일명: IT과학정보.docx
📁 위치: fileList/IT과학정보.docx
📝 요약: SK텔레콤이 SK AX와 협력하여 개발한 AI 업무 지원 시스템 '에이닷 비즈'가 연말까지 SK그룹 내 25개 사에 도입될 예정이다. 이 시스템은 자연어 처리를 통해 회의록 작성 시간을 60%, 보고서 작성 시간을 40% 단축시키며, 정보 검색, 일정 관리, 채용 등 다양한 업무를 지원한다. 에이전트 빌더와 스토어 기능을 통해 IT 지식이 없는 구성원도 쉽...
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

--- 4순위 (39.0%) ---
📄 파일명: 속도미달.docx
📁 위치: fileList/속도미달.docx
📝 요약

# 3D 시각화
전체 흐름   
Extractor: 질의에서 키워드 추출    
RAG Search: 키워드를 이용해 문서 검색   
Answer Generator: 검색 결과로부터 답변 생성    
Result Formatter: 결과 화면에 표시할 형식으로 가공   

전체 흐름 요약(Pipeline)
[extractor]    
    → 키워드 추출   
        → [rag_search]   
            → 문서 검색 + DB 조회 + 관련성 계산   
                → [answer_generator]   
                    → LLM 답변 생성   
                        → [result_formatter]   
                            → 최종 출력 포맷팅   
                                → END (완료)   

In [ ]:
# 노트북용 RAG + 3D 시각화 통합 코드
from typing import TypedDict, List, Dict, Any
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langgraph.graph import StateGraph, END
import pymysql

# 시각화 관련
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import datetime

# 상태 정의 (기존 구조 유지)
class AgentState(TypedDict):
    query: str
    keywords: str
    search_results: List[Dict[str, Any]]
    context: str
    result: str

# DB 접속 정보 (기존)
DB_CONFIG = {
    'host': 'localhost',
    'user': 'admin',
    'password': '1qazZAQ!',
    'db': 'final',
    'charset': 'utf8mb4'
}

# Chroma / LLM 설정 (기존)
CHROMA_PATH = "./rag_chroma/documents/title_summary/"
EMBEDDINGS = OllamaEmbeddings(model="exaone3.5:2.4b")
LLM = Ollama(model="exaone3.5:2.4b")
CHAT_LLM = ChatOllama(model="exaone3.5:2.4b", temperature=0.1)

# ----------------------------
# Notebook용 시각화 클래스
# ----------------------------
class RAGNotebookVisualizer:
    def __init__(self):
        # 에이전트 배치 (원래 구조/순서를 유지)
        self.agent_positions = {
            'extractor': (0.0, 0.0, 0.0),
            'rag_search': (2.0, 0.0, 1.0),
            'answer_generator': (4.0, 0.0, 2.0),
            'result_formatter': (6.0, 0.0, 1.0)
        }
        self.agent_names = list(self.agent_positions.keys())
        self.agent_status = {name: 'waiting' for name in self.agent_names}

        # 3D FigureWidget 생성
        self.fig3d = go.FigureWidget()
        self._init_3d_scene()

        # 관련성 막대 그래프
        self.bar_fig = go.FigureWidget(data=[go.Bar(x=[], y=[], text=[], textposition='auto')],
                                      layout=go.Layout(title='검색 결과 관련성', height=260, margin=dict(t=40)))
        # 진행률 바 및 로그
        self.progress = widgets.FloatProgress(value=0.0, min=0.0, max=100.0, description='진행률')
        self.log = widgets.Textarea(value='', placeholder='로그가 여기에 표시됩니다.', layout=widgets.Layout(width='100%', height='600px'))

        # 전체 레이아웃 (좌: 3D, 우: 차트/로그)
        left = widgets.VBox([self.fig3d], layout=widgets.Layout(width='65%'))
        right = widgets.VBox([self.bar_fig, self.progress, self.log], layout=widgets.Layout(width='35%'))
        self.container = widgets.HBox([left, right])

    def _init_3d_scene(self):
        # 노드 좌표
        xs = [self.agent_positions[n][0] for n in self.agent_names]
        ys = [self.agent_positions[n][1] for n in self.agent_names]
        zs = [self.agent_positions[n][2] for n in self.agent_names]

        # 에이전트 노드 trace
        node_trace = go.Scatter3d(
            x=xs, y=ys, z=zs,
            mode='markers+text',
            marker=dict(size=14, color=['gray'] * len(self.agent_names), opacity=0.9),
            text=self.agent_names,
            textposition="top center",
            name="Agents"
        )
        self.fig3d.add_trace(node_trace)

        # 에지(연결선) trace (순서대로 연결)
        for i in range(len(self.agent_names) - 1):
            a = self.agent_positions[self.agent_names[i]]
            b = self.agent_positions[self.agent_names[i+1]]
            edge = go.Scatter3d(
                x=[a[0], b[0]], y=[a[1], b[1]], z=[a[2], b[2]],
                mode='lines', line=dict(color='lightgray', width=4), showlegend=False
            )
            self.fig3d.add_trace(edge)

        # 검색 결과 placeholder trace (이후 업데이트)
        search_trace = go.Scatter3d(
            x=[], y=[], z=[],
            mode='markers+text',
            marker=dict(size=[], color=[], colorscale='Viridis', cmin=0, cmax=100, opacity=0.8, showscale=True,
                        colorbar=dict(title='관련성 (%)')),
            text=[], textposition='top center',
            name='Search Results'
        )
        self.fig3d.add_trace(search_trace)

        # 레이아웃 세팅
        self.fig3d.update_layout(
            title='RAG 처리 과정 3D 시각화 (Notebook)',
            scene=dict(
                xaxis=dict(title='단계'),
                yaxis=dict(title='데이터 흐름'),
                zaxis=dict(title='관련성 기반 높이')
            ),
            height=600,
            margin=dict(l=0, r=0, t=40, b=0)
        )

    def show(self):
        display(self.container)

    def _append_log(self, text: str):
        ts = datetime.datetime.now().strftime("%H:%M:%S")
        self.log.value += f"[{ts}] {text}\n"

    def update_agent_status(self, agent_name: str, status: str, data: List[Dict[str, Any]] = None):
        """
        status: 'processing', 'completed', 'error', 'waiting'
        data: optional, 검색 결과 전달 시 사용
        """
        # 로그
        self._append_log(f"{agent_name} -> {status}")

        # 상태 저장
        if agent_name in self.agent_status:
            self.agent_status[agent_name] = status

        # 색상 매핑
        def col(s):
            return {'processing': 'yellow', 'completed': 'green', 'error': 'red', 'waiting': 'gray'}.get(s, 'gray')

        colors = [col(self.agent_status[n]) for n in self.agent_names]
        # node 색 업데이트 (trace 0)
        self.fig3d.data[0].marker.color = colors

        # 진행률 업데이트
        completed = sum(1 for v in self.agent_status.values() if v == 'completed')
        self.progress.value = (completed / len(self.agent_status)) * 100

        # 검색 결과 전달 시 3D와 막대차트 업데이트
        if data is not None and agent_name == 'rag_search':
            self.update_search_results(data)

    def update_search_results(self, results: List[Dict[str, Any]]):
        """results: list of dicts with keys 'file_name' and 'relevance' (0-100) and optionally others"""
        # 정렬(이미 정렬되어 있을 수 있음)
        results = sorted(results, key=lambda r: r.get('relevance', 0), reverse=True)

        if not results:
            # clear traces
            # search trace는 마지막으로 추가된 trace 중 이름이 'Search Results' 인 것을 찾아 초기화
            for tr in self.fig3d.data:
                if getattr(tr, 'name', '') == 'Search Results':
                    tr.x = []; tr.y = []; tr.z = []; tr.marker.size = []; tr.marker.color = []; tr.text = []
            self.bar_fig.data[0].x = []; self.bar_fig.data[0].y = []; self.bar_fig.data[0].text = []
            return

        n = len(results)
        base_x, base_y, base_z = self.agent_positions['rag_search']

        xs, ys, zs, sizes, colors, texts = [], [], [], [], [], []
        radius = 1.5
        for i, r in enumerate(results):
            angle = (i * 2 * np.pi) / n
            x = base_x + radius * np.cos(angle)
            y = base_y + radius * np.sin(angle)
            z = base_z + (r.get('relevance', 0) / 100.0) * 2.0
            xs.append(x); ys.append(y); zs.append(z)
            sizes.append(max(6, r.get('relevance', 0) / 4.0))
            colors.append(r.get('relevance', 0))
            texts.append(f"{r.get('file_name', '파일')}: {r.get('relevance', 0)}%")

        # update search trace (찾아서 업데이트)
        for tr in self.fig3d.data:
            if getattr(tr, 'name', '') == 'Search Results':
                tr.x = xs
                tr.y = ys
                tr.z = zs
                tr.marker.size = sizes
                tr.marker.color = colors
                tr.text = texts
                break

        # update bar chart
        file_names = [r.get('file_name', '') for r in results]
        relevances = [r.get('relevance', 0) for r in results]
        self.bar_fig.data[0].x = file_names
        self.bar_fig.data[0].y = relevances
        self.bar_fig.data[0].text = [f"{v}%" for v in relevances]


# 전역 시각화 객체 (노트북에서 사용자가 show() 호출)
visualizer = RAGNotebookVisualizer()

# ----------------------------
# 기존 에이전트 함수들: 구조 변경 없이 시각화 업데이트만 추가
# ----------------------------

# 1. 키워드 추출 에이전트
def extractor_agent(state: AgentState):
    visualizer.update_agent_status('extractor', 'processing')
    keyword_prompt = PromptTemplate.from_template(
        """사용자의 질문에서 띄어쓰기 확인하고 찾고자 하는 키워드를 쉼표로 구분하여 출력하세요.
        벡터스토어 검색을 위한 최적의 키워드를 추출해주세요.
        \n질문: {query}"""
    )
    formatted_prompt = keyword_prompt.format(query=state["query"])
    keywords = LLM.invoke(formatted_prompt)
    visualizer.update_agent_status('extractor', 'completed')
    return {**state, "keywords": keywords.strip()}

# 2. RAG 검색 에이전트
def rag_search_agent(state: AgentState):
    visualizer.update_agent_status('rag_search', 'processing')
    try:
        vectorstore = Chroma(persist_directory=CHROMA_PATH, embedding_function=EMBEDDINGS)
        results = vectorstore.similarity_search_with_score(state["keywords"], k=10)

        if not results:
            visualizer.update_agent_status('rag_search', 'completed', [])
            return {**state, "search_results": [], "context": ""}

        best_doc_info = {}
        for doc, score in results:
            doc_id = doc.metadata.get("id")
            if doc_id:
                content = doc.page_content.strip()
                if not content:
                    continue
                if doc_id not in best_doc_info or score < best_doc_info[doc_id][0]:
                    best_doc_info[doc_id] = (score, content)

        if not best_doc_info:
            visualizer.update_agent_status('rag_search', 'completed', [])
            return {**state, "search_results": [], "context": ""}

        scores = [score for score, _ in best_doc_info.values()]
        min_score = min(scores)
        max_score = max(scores)

        conn = None
        search_results = []
        try:
            conn = pymysql.connect(**DB_CONFIG)
            cursor = conn.cursor(pymysql.cursors.DictCursor)

            for doc_id, (score, content) in best_doc_info.items():
                cursor.execute("SELECT * FROM documents WHERE id = %s", (doc_id,))
                row = cursor.fetchone()
                if not row:
                    continue

                if min_score == max_score:
                    relevance = 100.0
                else:
                    relevance = round((1 - (score - min_score) / (max_score - min_score)) * 100, 1)

                search_results.append({
                    "relevance": relevance,
                    "file_name": row["file_name"],
                    "file_location": row["file_location"],
                    "summary": row["summary"][:200] + "..." if row["summary"] else "요약 없음",
                    "title": row["title"],
                    "created_at": row["created_at"].strftime("%Y-%m-%d") if row["created_at"] else "N/A",
                    "doc_type": row["doc_type"],
                    "content": content
                })

            search_results.sort(key=lambda x: x["relevance"], reverse=True)
            search_results = search_results[:10]
            context = "\n\n".join([res['content'] for res in search_results])

        except Exception as e:
            print(f"❌ MySQL 오류: {e}")
            visualizer.update_agent_status('rag_search', 'error')
            return {**state, "search_results": [], "context": ""}
        finally:
            if conn:
                conn.close()

        visualizer.update_agent_status('rag_search', 'completed', search_results)
        return {**state, "search_results": search_results, "context": context}

    except Exception as e:
        print(f"❌ RAG 검색 오류: {e}")
        visualizer.update_agent_status('rag_search', 'error')
        return {**state, "search_results": [], "context": ""}

# 3. 답변 생성 에이전트
def answer_generator_agent(state: AgentState):
    visualizer.update_agent_status('answer_generator', 'processing')

    if not state["search_results"]:
        visualizer.update_agent_status('answer_generator', 'completed')
        return {**state, "result": "관련 정보를 찾을 수 없습니다."}

    try:
        prompt = ChatPromptTemplate.from_template(
            """다음 문서 내용들을 바탕으로 사용자 질문에 답변해 주세요.
            - 문서에 직접 언급된 내용만을 바탕으로 답변해야 합니다.
            - 정보가 없는 경우 "정보가 없습니다"라고 명확히 답변해 주세요.
            - 항상 공손하고 전문적인 어조를 유지해 주세요.

            ### 검색 결과 요약:
            {search_summary}

            ### 문서 내용:
            {context}

            ### 사용자 질문:
            {query}

            ### 답변:
            찾은 문서중에 몇번째 정보가 가장 관련이 높은지를 상세히 답변해 주세요.
            (몇번째, 무슨파일인지, 어디에 있는지, 요약은 무엇인지 등)
            """
        )

        search_summary = "\n".join([
            f"{i+1}순위 ({res['relevance']}%): {res['file_name']} ({res['doc_type']}) - {res['summary']}"
            for i, res in enumerate(state["search_results"])
        ])

        chain = prompt | CHAT_LLM | StrOutputParser()
        result = chain.invoke({
            "search_summary": search_summary,
            "context": state["context"],
            "query": state["query"]
        })

        visualizer.update_agent_status('answer_generator', 'completed')
        return {**state, "result": result.strip()}

    except Exception as e:
        print(f"❌ 답변 생성 오류: {e}")
        visualizer.update_agent_status('answer_generator', 'error')
        return {**state, "result": "답변을 생성하는 중에 오류가 발생했습니다."}

# 4. 결과 포맷팅 에이전트
def result_formatter_agent(state: AgentState):
    visualizer.update_agent_status('result_formatter', 'processing')

    if not state["search_results"]:
        visualizer.update_agent_status('result_formatter', 'completed')
        return state

    formatted_result = "\n🔍 검색 결과:\n"
    for i, res in enumerate(state["search_results"], 1):
        formatted_result += f"\n--- {i}순위 ({res['relevance']}%) ---\n"
        formatted_result += f"📄 파일명: {res['file_name']}\n"
        formatted_result += f"📁 위치: {res['file_location']}\n"
        formatted_result += f"📝 요약: {res['summary']}\n"
        formatted_result += f"🏷️ 유형: {res['doc_type']}\n"
        formatted_result += f"🕒 등록일: {res['created_at']}\n"

    formatted_result += f"\n💬 AI 답변:\n{state['result']}"

    visualizer.update_agent_status('result_formatter', 'completed')
    return {**state, "result": formatted_result}

# LangGraph 연결 (기존 구조 유지)
graph = StateGraph(AgentState)
graph.add_node("extractor", extractor_agent)
graph.add_node("rag_search", rag_search_agent)
graph.add_node("answer_generator", answer_generator_agent)
graph.add_node("result_formatter", result_formatter_agent)

graph.set_entry_point("extractor")
graph.add_edge("extractor", "rag_search")
graph.add_edge("rag_search", "answer_generator")
graph.add_edge("answer_generator", "result_formatter")
graph.add_edge("result_formatter", END)

app = graph.compile()

# 사용 예시 (노트북 셀에서 실행)
# 1) 먼저 시각화를 표시:
visualizer.show()
# 2) 그 다음 RAG 파이프라인 실행:
state = {"query": "홍민택 카카오", "keywords": "", "search_results": [], "context": "", "result": ""}
result = app.invoke(state)
print(result["result"])
#
# (visualizer는 에이전트 실행 도중 상태를 실시간으로 업데이트합니다.)

    'data': [{'marker': {'color': ['gray', 'gray', 'gray', 'gray'…


🔍 검색 결과:

--- 1순위 (100.0%) ---
📄 파일명: IT기사.docx
📁 위치: fileList/IT기사.docx
📝 요약: 홍민택 카카오 최고제품책임자(CPO)가 사내 공지를 통해 카카오톡 개편으로 인한 사용자 불만에 사과하며, 기존 서비스 유지와 함께 친구탭 피드 노출 방식 변경 및 인스타그램식 콘텐츠 분리 배치 계획을 밝혔다. 트래픽 감소는 없다고 전망하면서도 사용자 불편 최소화에 초점을 맞추겠다고 강조했다. 개편 배경은 카카오의 성장 정체 상황에서의 혁신 필요성과 기존 메...
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

--- 2순위 (64.8%) ---
📄 파일명: IT과학정보.docx
📁 위치: fileList/IT과학정보.docx
📝 요약: SK텔레콤이 SK AX와 협력하여 개발한 AI 업무 지원 시스템 '에이닷 비즈'가 연말까지 SK그룹 내 25개 사에 도입될 예정이다. 이 시스템은 자연어 처리를 통해 회의록 작성 시간을 60%, 보고서 작성 시간을 40% 단축시키며, 정보 검색, 일정 관리, 채용 등 다양한 업무를 지원한다. 에이전트 빌더와 스토어 기능을 통해 IT 지식이 없는 구성원도 쉽...
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

--- 3순위 (57.5%) ---
📄 파일명: 해외축구_기사.docx
📁 위치: fileList/해외축구_기사.docx
📝 요약: 아틀레티코 마드리드는 잉글랜드 출신 공격수 메이슨 그린우드를 영입하려는 의지를 보이며, 마르세유와는 이적료 약 1225억원으로 협상 중이다. 그린우드는 맨체스터 유나이티드 유스 시절부터 두각을 나타내다가 2021년 이후 법적 문제로 인해 위기를 겪었으나, 이후 마르세유로 이적해 리그앙 득점왕에 오르며 다시 주목받았다. 현재 그린우드의 기세는 이어지고 있어,...
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

--- 4순위 (32.6%) ---
📄 파일명: 속도미달.docx
📁 위치: fileList/속도미달.docx
📝 요약

: 